In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44987
2,Huelva,Confirmados PDIA 14 días,453
3,Huelva,Tasa PDIA 14 días,"88,27484069606564"
4,Huelva,Confirmados PDIA 7 días,184
5,Huelva,Tasa PDIA 7 dias,"35,85556443283902"
6,Huelva,Total Confirmados,45186
7,Huelva,Curados,42210
8,Huelva,Fallecidos,410


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44987.0


/tmp/ipykernel_5010/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12452.0


/tmp/ipykernel_5010/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_5010/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_5010/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_5010/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3688 personas en los últimos 7 días 

Un positivo PCR cada 1581 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44987.0,184.0,453.0,513170.0,35.855564,88.274841,82.0
Huelva-Costa,26633.0,134.0,325.0,289548.0,46.279028,112.243911,57.0
Huelva (capital),12452.0,39.0,91.0,143837.0,27.114025,63.266058,27.0
Condado-Campiña,13952.0,30.0,83.0,156231.0,19.202335,53.126460,19.0
Cartaya,1940.0,34.0,80.0,20083.0,169.297416,398.346861,9.0
Isla Cristina,3101.0,20.0,48.0,21393.0,93.488524,224.372458,7.0
Sierra de Huelva-Andévalo Central,3981.0,18.0,40.0,67391.0,26.709798,59.355107,6.0
Moguer,1935.0,7.0,15.0,21867.0,32.011707,68.596515,5.0
Ayamonte,2036.0,21.0,43.0,21104.0,99.507202,203.752843,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Nava (La),9.0,2.0,2.0,258.0,775.193798,775.193798,0.0
Cañaveral de León,33.0,1.0,3.0,396.0,252.525253,757.575758,NaN
San Silvestre de Guzmán,25.0,1.0,4.0,614.0,162.866450,651.465798,NaN
Cartaya,1940.0,34.0,80.0,20083.0,169.297416,398.346861,9.0
Berrocal,20.0,1.0,1.0,303.0,330.033003,330.033003,NaN
Aroche,128.0,2.0,9.0,3054.0,65.487885,294.695481,1.0
Sanlúcar de Guadiana,35.0,1.0,1.0,403.0,248.138958,248.138958,0.0
Higuera de la Sierra,65.0,1.0,3.0,1291.0,77.459334,232.378002,0.0
Cumbres Mayores,134.0,3.0,4.0,1749.0,171.526587,228.702115,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Punta Umbría,1255.0,1.0,9.0,15355.0,6.512537,58.612830,0.0,0.111111
San Juan del Puerto,758.0,1.0,8.0,9411.0,10.625863,85.006907,1.0,0.125000
Aroche,128.0,2.0,9.0,3054.0,65.487885,294.695481,1.0,0.222222
San Silvestre de Guzmán,25.0,1.0,4.0,614.0,162.866450,651.465798,0.0,0.250000
Bonares,455.0,1.0,4.0,6060.0,16.501650,66.006601,1.0,0.250000
Cañaveral de León,33.0,1.0,3.0,396.0,252.525253,757.575758,0.0,0.333333
Higuera de la Sierra,65.0,1.0,3.0,1291.0,77.459334,232.378002,0.0,0.333333
Condado-Campiña,13952.0,30.0,83.0,156231.0,19.202335,53.126460,19.0,0.361446
Almonte,2261.0,8.0,21.0,24507.0,32.643734,85.689803,5.0,0.380952
